In [ ]:
def _MA_screen(df, x):
    df['SMA'] = round(df['Adj Close'].rolling(window=x).mean(), 2)  # Simple Moving Average

    # TEMA triple exponential moving average | EMA Exponential Moving Average
    df['EMA1'] = round(df['Adj Close'].ewm(span=x, adjust=False).mean(), 2)
    df['EMA2'] = round(df['EMA1'].ewm(span=x, adjust=False).mean(), 2)
    df['EMA3'] = round(df['EMA2'].ewm(span=x, adjust=False).mean(), 2)
    df['TEMA'] = 3 * (df['EMA1'] - df['EMA2']) + df['EMA3']

    # up/down from previous 2 days
    df['SMA_2'] = df.SMA.shift(2)
    df['TEMA_2'] = df.TEMA.shift(2)
    df['SMA_up'] = df.apply(lambda x: '+' if x['SMA'] >= x['SMA_2'] else '-', axis=1)
    df['TEMA_up'] = df.apply(lambda x: '+' if x['TEMA'] >= x['TEMA_2'] else '-', axis=1)

    # reverse up/down
    df['SMA_up_1'] = df.SMA_up.shift(1)
    df['TEMA_up_1'] = df.TEMA_up.shift(1)
    df = df.dropna(subset=['SMA', 'SMA_2', 'TEMA', 'TEMA_2', 'SMA_up_1', 'TEMA_up_1'])
    df['SMA_up_rev'] = df.apply(lambda x: x['SMA_up'] if x['SMA_up'] != x['SMA_up_1'] else '', axis=1)
    df['TEMA_up_rev'] = df.apply(lambda x: x['TEMA_up'] if x['TEMA_up'] != x['TEMA_up_1'] else '', axis=1)

    SMA_seq = ''
    TEMA_seq = ''
    seq = 0
    for i in df.index[::-1]:  # look backwards
        seq += 1
        if bool(SMA_seq) & bool(TEMA_seq):
            break
        if (not SMA_seq) & bool(df['SMA_up_rev'][i]):
            SMA_seq = df['SMA_up_rev'][i] + str(seq)
        if (not TEMA_seq) & bool(df['TEMA_up_rev'][i]):
            TEMA_seq = df['TEMA_up_rev'][i] + str(seq)
    return [TEMA_seq, SMA_seq]

In [ ]:
# test _MA_screen with 1 symbol
import contextlib
import io

import pandas as pd
import yfinance as yf

yf.pdr_override()
pd.options.mode.chained_assignment = None  # default='warn'

with contextlib.redirect_stdout(io.StringIO()):
    df_yh = yf.download('NVDA', period="120d")
    # df_yh = yf.download('NVDA', period="20d", interval="1h")
    
_MA_screen(df_yh, 20)

In [ ]:
import contextlib
import io

import pandas as pd
import yfinance as yf

yf.pdr_override()
pd.options.mode.chained_assignment = None  # default='warn'

def _MA_screen_list(stockList, x):
    stockList = list(set(stockList))  # make elements unique
    stockList = sorted(stockList)
    rows = []
    for symbol in stockList:
        try:
            with contextlib.redirect_stdout(io.StringIO()):
                df_yh = yf.download(symbol, period="120d")
                # df_yh = yf.download(symbol, period="20d", interval="1h")
            row = _MA_screen(df_yh, x)
            row.append(symbol)
            rows.append(row)
        except Exception:
            print(symbol)
    out_df = pd.DataFrame(rows, columns=['TEMA_seq', 'SMA_seq', 'symbol'])

    out_df.TEMA_seq = out_df.TEMA_seq.astype(float)
    out_df.SMA_seq = out_df.SMA_seq.astype(float)
    out_df = out_df.sort_values(by=['TEMA_seq', 'SMA_seq'], ascending=False)

    print(out_df)

In [ ]:
stockList = ['ABNB', 'SPCE']
_MA_screen_list(stockList, 20)

In [ ]:
# Scan index companies
from pandas import DataFrame
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
lst = stock_data.get_stocks_by_index('DOW JONES')  # DOW JONES | S&P 500 | NASDAQ 100
df = DataFrame(lst)
stockList = df['symbol'].tolist()  # get list from a column

_MA_screen_list(stockList, 20)